# Retrieval-Augmented Generation (RAG)

## Wprowadzenie

Retrieval-Augmented Generation (RAG) to podejście łączące systemy wyszukiwania informacji z generatywnymi modelami językowymi (LLM). Pozwala to na tworzenie odpowiedzi, które są zarówno precyzyjne (dzięki zewnętrznemu źródłu wiedzy) jak i naturalnie sformułowane (dzięki zdolnościom generatywnym LLM).

### Dlaczego RAG jest istotny?

1. **Aktualność informacji** - LLM-y są trenowane na danych historycznych i nie posiadają wiedzy o wydarzeniach po dacie odcięcia treningu
2. **Redukcja konfabulacji** - Poprzez dostarczenie wiarygodnych źródeł, LLM ma mniejszą tendencję do wymyślania nieprawdziwych informacji
3. **Weryfikowalność** - Odpowiedzi mogą zawierać referencje do źródeł, co zwiększa ich wiarygodność
4. **Specjalistyczna wiedza** - Można podłączyć modele do specjalistycznych źródeł wiedzy bez konieczności ich dotrenowywania

## Komponenty systemu RAG

Typowy system RAG składa się z trzech głównych komponentów:

1. **Retriever - System wyszukiwania** - Odpowiedzialny za znalezienie odpowiednich informacji w źródle wiedzy
2. **Model generatywny** - Wykorzystuje znalezione informacje i zapytanie użytkownika do stworzenia odpowiedzi
3. **Żródło wiedzy** - Źródło informacji, które może być przeszukiwane

![RAG Architecture](assets/008-02.%20Architektura%20RAG.png)
Źródło: https://www.clarifai.com/blog/what-is-rag-retrieval-augmented-generation

### Przepływ informacji w RAG

1. Użytkownik zadaje pytanie
2. System wyszukiwania znajduje odpowiednie dokumenty/informacje związane z pytaniem
3. Znalezione informacje wraz z oryginalnym pytaniem są przekazywane do modelu generatywnego
4. Model generuje odpowiedź bazującą zarówno na pytaniu, jak i dostarczonych informacjach
5. Odpowiedź jest zwracana użytkownikowi, często z referencjami do źródeł

## Przykład prostego RAGa korzystającego z wyszukiwania informacji w Internecie

Poniżej zaimplementujemy prosty, ale skuteczny system RAG, który będzie:
- Wyszukiwał informacje w internecie przy pomocy DuckDuckGo
- Wykorzystywał LLMa do generowania odpowiedzi na podstawie znalezionych informacji

### Import bibliotek

In [ ]:
import os
from typing import List, Dict, Any
from openai import OpenAI
import time
from IPython.display import Markdown, display
# Biblioteka do wyszukiwania w DuckDuckGo (bardziej otwarta konkurencja dla Google)
from duckduckgo_search import DDGS
from pydantic import BaseModel, Field

### Konfiguracja klienta OpenAI

In [ ]:
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

### Implementacja komponentu wyszukiwania (Retriever)

W naszym przypadku, jako źródło wiedzy użyjemy internetu, a konkretnie wyszukiwarki DuckDuckGo. Stworzymy funkcję, która będzie wyszukiwać informacje na podstawie zapytania użytkownika.

In [ ]:
def retrieve_information(query: str, max_results: int = 5) -> List[Dict[str, Any]]:
    """
    Wyszukuje informacje związane z zapytaniem użytkownika za pomocą DuckDuckGo.
    
    Args:
        query: Zapytanie użytkownika
        max_results: Maksymalna liczba wyników do zwrócenia
        
    Returns:
        Lista słowników zawierających znalezione informacje
    """
    # Inicjalizacja wyszukiwarki DuckDuckGo
    ddgs = DDGS()
    
    # Wykonanie wyszukiwania
    results = list(ddgs.text(query, max_results=max_results))
    
    # Dodanie pola id zawierającego liczbę porządkową do każdego elementu
    for i, result in enumerate(results, 1):
        result['id'] = i
    
    return results

In [ ]:
retrieve_information("szef kuchni")

### Implementacja komponentu generatywnego (Generator)

Teraz stworzymy funkcję, która wykorzysta model GPT-4o do wygenerowania odpowiedzi na podstawie pytania użytkownika i znalezionych informacji.

In [ ]:
def generate_answer(query: str, retrieved_info: List[Dict[str, Any]]) -> str:
    """
    Generuje odpowiedź na podstawie zapytania i znalezionych informacji.
    
    Args:
        query: Zapytanie użytkownika
        retrieved_info: Lista słowników zawierających znalezione informacje
        
    Returns:
        Wygenerowana odpowiedź
    """

    # Przygotowanie kontekstu z wyszukanych informacji
    context = "\n\n".join([
        f"Tytuł: {result.get('title', 'Brak tytułu')}\n"
        f"Opis: {result.get('body', 'Brak opisu')}\n"
        f"Źródło: {result.get('href', 'Brak źródła')}"
        for result in retrieved_info
    ])
    
    # Instrukcje dla modelu
    system_prompt = f"""
    Jesteś pomocnym asystentem, który odpowiada na pytania użytkownika w oparciu o dostarczone konteksty.
    Twoim zadaniem jest:
    1. Przeanalizować dostarczone informacje i wybrać te, które są najbardziej istotne dla pytania.
    2. Sformułować zwięzłą, ale kompletną odpowiedź opartą na tych informacjach (i tylko na nich - nie uzupełniaj własną wiedzą). 
    3. Gdy informacje są sprzeczne lub niepewne, zaznaczyć to w odpowiedzi.
    4. Dostosować czas odpowiedzi do aktualnej daty i godziny - jeśli mówisz o wydarzeniach z przeszłości, użyj czasu przeszłego, a jeśli o przyszłości - czasu przyszłego.
    5. Podać źródła użytych informacji w formie odnośników [1], [2], itp. na końcu odpowiedzi. Numer powinien odpowiadać wartośći pola 'id' źródła w kontekście.
    6. Jeśli dostarczone informacje nie pozwalają na odpowiedź, przyznaj to uczciwie.
    7. Odpowiedź w tym samym języku, w którym zadano pytanie, Nnawet jeśli część źródeł jest w innym języku.
    
    Odpowiedź powinna być napisana w języku pytania, prostym do zrozumienia, i dobrze zorganizowana.

    Dodatkowe informacje:
    - Aktualna data i czas: {time.strftime('%Y-%m-%d %H:%M:%S')}. Uwzględnij tę informację tworząc odpowiedź.
    """
    
    prompt = f"""
    Pytanie:
    {query}
    ---
    Kontekst:
    {context}
    """

    # Przygotowanie wiadomości dla API
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]
    
    # Wywołanie API
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0.2,  # Niska temperatura dla bardziej deterministycznych odpowiedzi
        max_tokens=5000  # Limit długości odpowiedzi
    )
    
    return response.choices[0].message.content

### Łączenie komponentów w kompletny system RAG

Teraz połączymy wszystkie komponenty, aby stworzyć kompletny system RAG.

In [ ]:
def rag_system_simple(query: str, max_results: int = 5):
    """
    Kompletny system RAG, który wyszukuje informacje i generuje odpowiedź.
    
    Args:
        query: Zapytanie użytkownika
        max_results: Maksymalna liczba wyników wyszukiwania
        
    Returns:
        Wygenerowana odpowiedź
    """
    print(f"Zapytanie: {query}")
    print("Wyszukiwanie informacji...")
    start_time = time.time()
    
    # Wyszukiwanie informacji
    retrieved_info = retrieve_information(query, max_results)
    retrieval_time = time.time() - start_time
    print(f"Znaleziono {len(retrieved_info)} wyników w {retrieval_time:.2f} sekund.")
    
    print("Generowanie odpowiedzi...")
    start_time = time.time()
    
    # Generowanie odpowiedzi
    answer = generate_answer(query, retrieved_info)
    generation_time = time.time() - start_time
    print(f"Odpowiedź wygenerowana w {generation_time:.2f} sekund.")
    
    # Informacje o źródłach
    print("\nŹródła informacji:")
    for i, result in enumerate(retrieved_info, 1):
        print(f"[{result.get('id')}] {result.get('title')}: {result.get('href')}")
    
    return answer

### Funkcja pomocnicza do wyświetlania wyników

In [ ]:
def display_answer(answer):
    """
    Wyświetla odpowiedź w ładnym formacie Markdown.
    
    Args:
        answer: Wygenerowana odpowiedź
    """
    display(Markdown("---\n## Odpowiedź:\n" + answer))

### Przykładowe wywołania

Przetestujmy, czy to działa

In [ ]:
# Przykład 1: Zapytanie o aktualne wydarzenie
query1 = "Kiedy i gdzie odbyły się Igrzyska Olimpijskie 2024?"
answer1 = rag_system_simple(query1, max_results=10)
display_answer(answer1)

In [ ]:
# Przykład 2: Zapytanie techniczne
query2 = "Który model od Open AI jest najnowszy? Czym się różni od wcześniejszych modeli?"
answer2 = rag_system_simple(query2, max_results=10)
display_answer(answer2)

In [ ]:
# Przykład 3: Zapytanie o osobę
query3 = "Kto obecnie jest premierem Polski i jakie ma doświadczenie polityczne?"
answer3 = rag_system_simple(query3, max_results=10)
display_answer(answer3)

## Trochę bardziej zaawansowany RAG

W bardziej zaawansowanej wersji RAG, zamiast używać pojedynczego zapytania wyszukiwania, i to będącego pytaniem użytkownika, możemy wygenerować kilka różnych zapytań, które lepiej uchwycą kontekst i intencję pytania użytkownika. To podejście, znane jako Multi-Query RAG, pomaga:

1. **Uzyskać szerszy kontekst** - Różne warianty zapytań mogą wydobyć informacje, które nie byłyby dostępne przy pojedynczym zapytaniu
2. **Lepiej radzić sobie z niejednoznacznościami** - Generując zapytania uwzględniające różne interpretacje pytania użytkownika
3. **Zwiększyć prawdopodobieństwo znalezienia odpowiednich informacji** - Więcej zapytań = większa szansa na trafienie w źródła zawierające potrzebne dane

Poniżej zaimplementujemy system RAG z generowaniem wielu zapytań wyszukiwania, używając modelu GPT-4o do stworzenia optymalnych haseł wyszukiwania na podstawie pierwotnego pytania użytkownika.

### Funkcja tworząca zapytania dla polecenia użytkownika

In [ ]:
def generate_search_queries(question: str, num_queries: int = 3) -> list[str]:
    """
    Generuje zoptymalizowane zapytania wyszukiwania dla DuckDuckGo na podstawie pytania użytkownika.
    
    Args:
        question: Pytanie lub prośba użytkownika
        num_queries: Liczba zapytań wyszukiwania do wygenerowania (domyślnie 3)
        
    Returns:
        Lista 'num_queries' terminów wyszukiwania
    """
    system_prompt = f"""
    Jesteś ekspertem w tworzeniu efektywnych zapytań wyszukiwania.
    Twoim zadaniem jest wygenerowanie zapytań do wyszukiwarki DuckDuckGo, 
    które pozwolą zebrać najbardziej przydatne informacje do odpowiedzi na pytanie użytkownika.
    
    Zapytania powinny:
    1. Być zwięzłe i konkretne
    2. Zawierać kluczowe słowa i frazy
    3. Obejmować różne aspekty pytania lub tematu
    4. Być sformułowane w taki sposób, aby zmaksymalizować trafność wyników wyszukiwania
    5. Pozwolić na wyszukiwanie informacji w różnych źródłach, nie tylko w encyklopediach czy artykułach naukowych
    6. Pozwolić na wyszukanie informacji w języku pytania użytkownika, oraz w języku angielskim
    
    Dodatkowe informacje:
    - Aktualna data i czas: {time.strftime('%Y-%m-%d %H:%M:%S')}. Uwzględnij tę informację tworząc odpowiedź.
    """
    
    user_prompt = f"""
    Stwórz dokładnie {num_queries} zapytań wyszukiwania do DuckDuckGo dla następującego pytania:
    ---
    {question}
    """
    
    class SearchQueries(BaseModel):
        """Klasa pydanticowa dla zapytań wyszukiwania wygenerowanych na podstawie pytania użytkownika."""
        queries: list[str] = Field(description=f"Lista dokładnie {num_queries} zapytań wyszukiwania do użycia z DuckDuckGo")
    
    # Wywołanie modelu GPT-4o
    response = client.beta.chat.completions.parse(
        model="gpt-4o",
        response_format=SearchQueries,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.2  # Niska temperatura dla bardziej deterministycznych odpowiedzi
    )
    
    return response.choices[0].message.parsed.queries

Zobaczmy czy to działa

In [ ]:
generate_search_queries("Jakie są najnowsze osiągnięcia w dziedzinie sztucznej inteligencji?", num_queries=10)

### RAG uwzględniający wyszukiwania wieloma hasłami

In [ ]:
def rag_system_multiquery(query: str, max_results_per_query: int = 3, num_queries: int = 3):
    """
    Zaawansowany system RAG wykorzystujący wielokrotne zapytania wyszukiwania.
    
    Args:
        query: Zapytanie użytkownika
        max_results_per_query: Maksymalna liczba wyników wyszukiwania dla każdego z wygenerowanych zapytań
        num_queries: Liczba zapytań wyszukiwania do wygenerowania (domyślnie 3)
        
    Returns:
        Wygenerowana odpowiedź
    """
    print(f"Zapytanie użytkownika: {query}")
    print(f"Generowanie {num_queries} optymalnych zapytań wyszukiwania...")
    
    # Generowanie zapytań do wyszukiwania
    search_queries = generate_search_queries(question=query, num_queries=num_queries)
    print(f"Wygenerowano {len(search_queries)} zapytań wyszukiwania:")
    for idx, search_query in enumerate(search_queries, 1):
        print(f"  {idx}. {search_query}")
    
    # Wyszukiwanie informacji dla każdego zapytania
    print("\nWyszukiwanie informacji...")
    start_time = time.time()
    all_results = []
    
    # Słownik do śledzenia liczby wyników dla każdego zapytania
    results_per_query = {}
    
    for i, search_query in enumerate(search_queries, 1):
        results = retrieve_information(search_query, max_results=max_results_per_query)
        # Dodanie informacji o zapytaniu wyszukiwania do każdego wyniku
        for result in results:
            result['search_query'] = search_query
        results_per_query[search_query] = len(results)        
        all_results.extend(results)
    
    retrieval_time = time.time() - start_time
    print(f"Znaleziono łącznie {len(all_results)} wyników w {retrieval_time:.2f} sekund.")
    
    # Wypisanie liczby wyników dla każdego zapytania
    print("\nLiczba wyników dla poszczególnych zapytań:")
    for i, (query, count) in enumerate(results_per_query.items(), 1):
        print(f"  [{i}] {query}: {count} wyników")
    
    # Usuwanie duplikatów na podstawie URL i treści
    unique_content = set()
    unique_results = []
    
    for result in all_results:
        # Tworzenie unikalnego identyfikatora na podstawie URL i fragmentu treści
        # Używamy tylko pierwszych 100 znaków treści, aby uniknąć nadmiernego zużycia pamięci
        content_id = (result['href'], result.get('body', '')[:100])
        
        if content_id not in unique_content:
            unique_content.add(content_id)
            unique_results.append(result)
    
    # Reset ID dla unikalnych wyników
    for i, result in enumerate(unique_results, 1):
        result['id'] = i
    
    print(f"Po usunięciu duplikatów pozostało {len(unique_results)} unikalnych wyników.")
    
    # Generowanie odpowiedzi
    print("\nGenerowanie odpowiedzi na podstawie znalezionych informacji...")
    start_time = time.time()
    answer = generate_answer(query, unique_results)
    generation_time = time.time() - start_time
    print(f"Odpowiedź wygenerowana w {generation_time:.2f} sekund.")
    
    # Informacje o źródłach
    print("\nŹródła informacji:")
    for result in unique_results:
        print(f"[{result['id']}] {result['title']}: {result['href']}")
        print(f"    Znaleziono przez zapytanie: {result.get('search_query', 'nieznane zapytanie')}")
    
    return answer

### Przykłady wywołań

In [ ]:
# Przykład 1: Zapytanie o aktualne wydarzenie
query1 = "Kiedy i gdzie odbyły się Igrzyska Olimpijskie 2024? O czym mówiły media?"
answer1 = rag_system_multiquery(query1, max_results_per_query=5, num_queries=5)
display_answer(answer1)

In [ ]:
# Przykład 2: Zapytanie techniczne
query2 = "Który model od Open AI jest najnowszy? Czym się różni od wcześniejszych modeli?"
answer2 = rag_system_multiquery(query2, max_results_per_query=5, num_queries=5)
display_answer(answer2)

In [ ]:
# Przykład 3: Zapytanie o osobę
query3 = "Kto obecnie jest premierem Polski i jakie ma doświadczenie polityczne?"
answer3 = rag_system_multiquery(query3, max_results_per_query=5, num_queries=5)
display_answer(answer3)

## RAG wykorzystujący indeks wektorowy

W poprzednich przykładach używaliśmy wyszukiwania w internecie jako źródła wiedzy. Alternatywnym, często bardziej efektywnym podejściem jest wykorzystanie bazy wektorowej do przechowywania i wyszukiwania dokumentów.

Podejście oparte o indeksy / bazy wektorowe ma kilka zalet:

1. **Wyszukiwanie semantyczne** - Zamiast dopasowywać słowa kluczowe, wyszukujemy dokumenty o podobnym znaczeniu
2. **Szybkość** - Lokalne przechowywanie dokumentów umożliwia szybkie wyszukiwanie bez opóźnień związanych z zapytaniami do zewnętrznych API
3. **Kontrola nad danymi** - Mamy pełną kontrolę nad tym, jakie dokumenty są w bazie wiedzy
4. **Prywatność** - Dokumenty i zapytania nie muszą opuszczać naszego środowiska

Poniżej zaimplementujemy prosty system RAG wykorzystujący bazę wektorową FAISS i model sentence-transformers do tworzenia wektorów (embeddings).

### Import bibliotek dla wersji z indeksem wektorowym

In [ ]:
import numpy as np
import faiss  # Biblioteka do efektywnego wyszukiwania wektorów
from sentence_transformers import SentenceTransformer  # Do tworzenia wektorów tekstu
import time

### Przygotowanie przykładowych dokumentów

W prawdziwym systemie te dokumenty mogłyby pochodzić z różnych źródeł, takich jak bazy wiedzy, strony internetowe, PDF-y, itp.
Na potrzeby tego przykładu stworzymy prosty zbiór dokumentów.

In [ ]:
# Przykładowe dokumenty dla naszej bazy wiedzy

documents = [
    # Dokumenty z kategorii sztuczna inteligencja
    "Sztuczna inteligencja (AI) to dziedzina informatyki zajmująca się tworzeniem systemów zdolnych do wykonywania zadań wymagających inteligencji ludzkiej.",
    "Uczenie maszynowe to podzbiór sztucznej inteligencji, który umożliwia systemom komputerowym uczenie się z danych bez wyraźnego programowania.",
    "Deep learning to technika uczenia maszynowego wykorzystująca sieci neuronowe z wieloma warstwami (głębokie sieci neuronowe) do analizy różnych czynników w danych.",
    "Natural Language Processing (NLP) to dziedzina AI zajmująca się interakcją między komputerami a językiem ludzkim.",
    "Computer Vision to dziedzina AI, która trenuje komputery do interpretowania i rozumienia świata wizualnego.",
    "Reinforcement learning to obszar uczenia maszynowego dotyczący podejmowania decyzji przez agenta w środowisku w celu maksymalizacji skumulowanej nagrody.",
    "Transformer to architektura sieci neuronowych oparta na mechanizmie uwagi, który poprawia wydajność w zadaniach przetwarzania języka naturalnego.",
    "Model GPT-4o, wydany przez OpenAI w 2024 roku, to multimodalny model AI, który rozumie tekst i obrazy.",
    "Systemy rekomendacyjne to aplikacje AI, które przewidują preferencje użytkowników i sugerują odpowiednie produkty lub treści.",
    "Etyka AI dotyczy etycznych implikacji i wyzwań związanych z rozwojem i wdrażaniem systemów sztucznej inteligencji.",
    "OpenAI to firma badawcza zajmująca się sztuczną inteligencją, znana z modeli takich jak GPT-3, GPT-4, DALL-E i Sora.",
    "Sora to model AI od OpenAI zaprezentowany w 2024 roku, który potrafi generować realistyczne filmy wideo na podstawie instrukcji tekstowych.",
    "Anthropic to firma AI założona przez byłych pracowników OpenAI, która stworzyła asystenta Claude.",
    "Claude to duży model językowy (LLM) od Anthropic, konkurencyjny wobec modeli GPT, znany z funkcji Constitutional AI mających zapewnić bezpieczeństwo.",
    "Gemini to model multimodalny od Google DeepMind, wcześniej znany jako Bard, wprowadzony jako konkurent ChatGPT.",
    "Meta (wcześniej Facebook) rozwija własne modele AI, w tym Llama, który został udostępniony jako open source dla społeczności badawczej.",
    "Hallucynacje AI to zjawisko, w którym modele generatywne tworzą nieprawdziwe informacje, które wydają się wiarygodne, ale nie mają potwierdzenia w faktach.",
    "RAG (Retrieval-Augmented Generation) to metoda łącząca wyszukiwanie informacji z generatywnymi modelami językowymi w celu poprawy dokładności odpowiedzi.",
    "Modele foundation to duże modele AI trenowane na ogromnych zbiorach danych, które mogą być dostosowywane do różnych zadań poprzez fine-tuning.",
    "Fine-tuning to proces dostosowywania wstępnie wytrenowanego modelu AI do konkretnego zadania lub domeny poprzez dodatkowe szkolenie na specyficznych danych.",
    "Prompt engineering to sztuka konstruowania efektywnych poleceń dla modeli AI w celu uzyskania pożądanych wyników.",
    "AGI (Artificial General Intelligence) odnosi się do hipotetycznej sztucznej inteligencji, która posiada zdolność rozumienia, uczenia się i wykonywania dowolnego zadania intelektualnego, które może wykonać człowiek.",
    "Transfer learning to technika uczenia maszynowego, w której model opracowany dla jednego zadania jest ponownie wykorzystywany jako punkt wyjścia dla modelu w drugim zadaniu.",
    "Embeddingi to reprezentacje wektorowe słów, zdań lub innych danych, które wychwytują znaczenie semantyczne i są używane w wielu zastosowaniach AI.",
    "Attention mechanism to kluczowy komponent architektur transformerowych, który pozwala modelom nadawać różne wagi różnym częściom danych wejściowych.",
    "GPT (Generative Pre-trained Transformer) to rodzina modeli językowych opracowanych przez OpenAI, które wykorzystują architektury transformerowe do generowania tekstu.",
    "BERT (Bidirectional Encoder Representations from Transformers) to model językowy opracowany przez Google, który reprezentuje kontekst słowa w obu kierunkach.",
    "Zero-shot learning to zdolność modelu AI do wykonywania zadań bez wcześniejszego trenowania na przykładach tego konkretnego zadania.",
    "Federated learning to technika uczenia maszynowego, w której model jest trenowany na wielu urządzeniach lub serwerach bez wymiany danych, chroniąc prywatność.",
    "Explainable AI (XAI) to podejście do sztucznej inteligencji, które koncentruje się na tworzeniu modeli, których działania mogą być zrozumiane przez ludzi.",
    
    # Dokumenty z kategorii cyberbezpieczeństwo
    "Cyberbezpieczeństwo to praktyka ochrony systemów, sieci i programów przed cyfrowymi atakami mającymi na celu dostęp, zmianę lub zniszczenie wrażliwych informacji.",
    "Malware to złośliwe oprogramowanie, które celowo wyrządza szkody w komputerach, serwerach lub sieciach komputerowych, obejmujące wirusy, trojany i ransomware.",
    "Phishing to cyberatak, w którym atakujący maskują się jako zaufana jednostka, aby nakłonić ofiary do ujawnienia poufnych danych, takich jak hasła czy dane karty kredytowej.",
    "Firewall to system bezpieczeństwa sieciowego, który monitoruje i kontroluje przychodzący i wychodzący ruch sieciowy zgodnie z określonymi zasadami bezpieczeństwa.",
    "VPN (Virtual Private Network) to technologia, która tworzy bezpieczne, szyfrowane połączenie przez mniej bezpieczną sieć, taką jak internet.",
    "Szyfrowanie to proces kodowania informacji w taki sposób, aby tylko upoważnione osoby miały do nich dostęp za pomocą klucza szyfrowania.",
    "Zero-day exploit to atak wykorzystujący nieznane wcześniej luki w oprogramowaniu, o których programiści nie wiedzą i nie mają dla nich łatek.",
    "SIEM (Security Information and Event Management) to narzędzie zapewniające analizę alertów bezpieczeństwa generowanych przez aplikacje i urządzenia sieciowe.",
    "Dwuskładnikowe uwierzytelnianie (2FA) to metoda bezpieczeństwa, która wymaga dwóch niezależnych form identyfikacji do uzyskania dostępu do zasobów.",
    "SOC (Security Operations Center) to zespół ekspertów ds. bezpieczeństwa, którzy monitorują i analizują środowisko bezpieczeństwa organizacji.",
    "CSRF (Cross-Site Request Forgery) to atak, który wymusza na zalogowanym użytkowniku wykonanie niechcianych działań na stronie internetowej.",
    "XSS (Cross-Site Scripting) to luka w zabezpieczeniach, która umożliwia atakującym wstrzykiwanie złośliwych skryptów do stron internetowych oglądanych przez innych użytkowników.",
    "SQL injection to technika ataku, w której złośliwy kod SQL jest wprowadzany do pól wejściowych formularza w celu uzyskania dostępu do bazy danych.",
    "Ransomware to rodzaj złośliwego oprogramowania, które blokuje dostęp do plików lub systemów komputerowych i żąda okupu za ich odblokowanie.",
    "DDoS (Distributed Denial of Service) to atak, w którym wiele systemów przytłacza cel (jak serwer, strona internetowa lub inna sieć) generując ruch internetowy.",
    "Pentesting (testowanie penetracyjne) to symulowany atak na system komputerowy w celu oceny bezpieczeństwa systemu i identyfikacji luk.",
    "Biometria to metoda uwierzytelniania oparta na unikalnych cechach fizycznych, takich jak odciski palców, rozpoznawanie twarzy czy skanowanie tęczówki.",
    "OSINT (Open-Source Intelligence) to zbieranie informacji z publicznie dostępnych źródeł do celów analizy bezpieczeństwa.",
    "APT (Advanced Persistent Threat) to długotrwały, ukierunkowany cyberatak, w którym atakujący zyskuje nieautoryzowany dostęp do sieci i pozostaje niewykryty przez dłuższy czas.",
    "Blockchain to technologia rozproszonej księgi, która zwiększa bezpieczeństwo poprzez przechowywanie danych w blokach połączonych kryptograficznie.",
    
    # Dokumenty z kategorii frontend
    "HTML (HyperText Markup Language) to standardowy język znaczników używany do tworzenia stron internetowych i aplikacji webowych.",
    "CSS (Cascading Style Sheets) to język arkuszy stylów używany do opisywania prezentacji dokumentu napisanego w języku znaczników, takim jak HTML.",
    "JavaScript to wysokopoziomowy, dynamicznie typowany język programowania, który jest jednym z podstawowych technologii World Wide Web.",
    "React to biblioteka JavaScript stworzona przez Facebooka do budowania interfejsów użytkownika, szczególnie aplikacji jednostronicowych.",
    "Angular to platforma TypeScript i framework do budowania aplikacji webowych i mobilnych, rozwijana przez Google.",
    "Vue.js to progresywny framework JavaScript do budowania interfejsów użytkownika, zaprojektowany by być przyswajany stopniowo.",
    "TypeScript to silnie typowany nadzbiór JavaScript, który kompiluje się do czystego JavaScript, opracowany przez Microsoft.",
    "Webpack to narzędzie do pakowania modułów dla nowoczesnych aplikacji JavaScript, przetwarzające aplikację i budujące graf zależności.",
    "Responsive Web Design to podejście do projektowania stron internetowych, które sprawia, że strony dobrze wyświetlają się na urządzeniach o różnych rozmiarach i orientacjach.",
    "PWA (Progressive Web App) to typ aplikacji dostarczanej za pośrednictwem sieci, ale oferującej funkcje podobne do natywnych aplikacji mobilnych.",
    "Babel to kompilator JavaScript, który przekształca nowoczesny kod JavaScript w wersję kompatybilną wstecz dla starszych przeglądarek.",
    "Redux to kontener stanu przewidywalnego dla aplikacji JavaScript, często używany z Reactem do zarządzania stanem aplikacji.",
    "SASS (Syntactically Awesome Style Sheets) to preprocesor CSS, który rozszerza możliwości CSS o zmienne, zagnieżdżone reguły i inne funkcje.",
    "Tailwind CSS to framework CSS oparty na klasach narzędziowych, który promuje szybkie tworzenie i dostosowywanie interfejsów bez opuszczania HTML.",
    "Material Design to język projektowania opracowany przez Google, który syntetyzuje klasyczne zasady dobrego designu z innowacjami technologii i nauki.",
    "SVG (Scalable Vector Graphics) to format grafiki wektorowej oparty na XML, używany do tworzenia dwuwymiarowych grafik z obsługą interaktywności i animacji.",
    "AJAX (Asynchronous JavaScript and XML) to zestaw technik programowania wykorzystywanych do tworzenia asynchronicznych aplikacji internetowych.",
    "SPA (Single Page Application) to aplikacja internetowa lub strona, która dynamicznie aktualizuje bieżącą stronę zamiast ładować całe nowe strony z serwera.",
    "SEO (Search Engine Optimization) to proces zwiększania jakości i ilości ruchu na stronie internetowej z organicznych wyników wyszukiwania.",
    "WebSocket to protokół komunikacyjny zapewniający dwukierunkowy kanał komunikacji przez pojedyncze połączenie TCP.",
    
    # Dokumenty z kategorii języki backendu
    "Node.js to środowisko uruchomieniowe JavaScript, które pozwala na wykonywanie kodu JavaScript poza przeglądarką, używane głównie do budowania szybkich aplikacji sieciowych.",
    "Python to interpretowany, wysokopoziomowy i ogólnego przeznaczenia język programowania, znany z czytelnej składni i szerokiego zastosowania od web developmentu po naukę o danych i AI.",
    "Java to wieloplatformowy, obiektowy język programowania, szeroko stosowany do tworzenia aplikacji enterprise i systemów korporacyjnych.",
    "C# to obiektowy język programowania opracowany przez Microsoft, używany głównie do tworzenia aplikacji Windows i aplikacji webowych z .NET Framework.",
    "PHP to skryptowy język programowania zaprojektowany specjalnie do tworzenia stron internetowych i aplikacji webowych.",
    "Ruby to dynamiczny, open-source język programowania z naciskiem na prostotę i produktywność, często używany z frameworkiem Rails do tworzenia aplikacji webowych.",
    "Go (Golang) to język programowania stworzony przez Google, zaprojektowany do budowania prostego, niezawodnego i wydajnego oprogramowania.",
    "Rust to wieloparadygmatowy język programowania systemowego skupiony na bezpieczeństwie pamięci bez użycia garbage collectora.",
    "SQL (Structured Query Language) to standardowy język używany do komunikacji z relacyjnymi bazami danych, umożliwiający manipulację danymi i ich zapytania.",
    "MongoDB to baza danych NoSQL typu document-store, która przechowuje dane w elastycznych dokumentach podobnych do JSON, zamiast w tabelach relacyjnych.",
    "Redis to sklep struktur danych typu open source (in-memory), używany jako baza danych, pamięć podręczna i broker wiadomości.",
    "GraphQL to język zapytań API i środowisko uruchomieniowe do realizacji tych zapytań z istniejącymi danymi, oferujący bardziej efektywne i elastyczne podejście niż REST.",
    "Django to wysokopoziomowy framework internetowy w Pythonie, który zachęca do szybkiego rozwoju i czystego, pragmatycznego projektowania.",
    "Express.js to minimalistyczny i elastyczny framework webowy dla Node.js, zapewniający zestaw funkcji do tworzenia aplikacji webowych i mobilnych.",
    "Spring Boot to framework oparty na Javie, który upraszcza proces tworzenia aplikacji opartych na Spring, minimalizując konfigurację.",
    "Laravel to framework aplikacji webowych z elegancką składnią dla PHP, ułatwiający typowe zadania programistyczne.",
    "Ruby on Rails to framework aplikacji webowych napisany w Ruby, który następuje zasadzie 'convention over configuration' (konwencja przed konfiguracją).",
    "ASP.NET Core to darmowy, open-source, cross-platform framework do budowania aplikacji internetowych na platformę .NET.",
    "RESTful API to interfejs programistyczny aplikacji (API) zgodny z ograniczeniami architektury REST i umożliwiający interakcję z usługami webowymi.",
    "Microservices to architektoniczny styl, w którym złożone aplikacje są podzielone na małe, niezależne usługi, które komunikują się przez sieć."
]

# Dodanie metadanych do dokumentów
document_metadata = []
for i, doc in enumerate(documents):
    category = ""
    if i < 30:
        category = "Sztuczna inteligencja"
    elif i < 50:
        category = "Cyberbezpieczeństwo"
    elif i < 70:
        category = "Frontend"
    else:
        category = "Backend"
        
    document_metadata.append({
        "id": i,
        "content": doc,
        "source": f"Dokument {i}",
        "category": category
    })

documents = document_metadata

### Implementacja systemu wektorowego RAG

Teraz zaimplementujemy poszczególne komponenty systemu RAG opartego o bazę wektorową jako osobne funkcje.

#### Wyznaczenie embeddingów dla każdego dokumentu

In [ ]:
# Inicjalizacja modelu do tworzenia embeddingów
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Wyodrębniamy same treści dokumentów
texts = [doc["content"] for doc in documents]

# Generujemy wektory dla każdego dokumentu
embeddings = embedding_model.encode(texts)



#### Inicjalizacja indeksu wektorowego (i dodanie do niego embeddingów domkumetnów)

Nie do końca moemy mówić tutaj o bazie wektorowej, bo FAISS nie przechowuje treści dokumentów. Jedynie pozwala nam efektywnie wyszukiwać indeksy najbardziej podobnych wektorów.

In [ ]:
# Wymiary wektorów
dimension = embeddings.shape[1]

# Inicjalizacja indeksu FAISS
index = faiss.IndexFlatIP(dimension)
faiss.normalize_L2(embeddings)

# Dodanie wektorów do indeksu
index.add(np.array(embeddings).astype('float32'))

print(f"Utworzono bazę wektorową z {len(documents)} dokumentami o wymiarze {dimension}")

#### Wyszukiwanie dokumentów w indeksie wektorowym

In [ ]:
def retrieve_documents_vector(query, documents, embedding_model, index, top_k=3):
    """
    Wyszukuje najbardziej podobne dokumenty do zapytania.
    
    Args:
        query: Zapytanie użytkownika
        documents: Lista dokumentów z metadanymi
        embedding_model: Model do generowania wektorów tekstu
        index: Indeks FAISS bazy wektorowej
        top_k: Liczba najbardziej podobnych dokumentów do zwrócenia
        
    Returns:
        Lista słowników zawierających najbardziej podobne dokumenty
    """
    # Generowanie wektora dla zapytania
    query_embedding = embedding_model.encode([query])
    faiss.normalize_L2(query_embedding)
    
    # Wyszukanie najbliższych wektorów
    similarities, indices = index.search(np.array(query_embedding).astype('float32'), top_k)
    
    # Pobranie odpowiadających dokumentów
    retrieved_docs = []
    for i, idx in enumerate(indices[0]):
        if idx < len(documents):  # Sprawdzenie, czy indeks jest prawidłowy
            doc = documents[idx].copy()
            doc["cosine_similarity"] = float(similarities[0][i])  # Podobieństwo kosinusowe
            doc["distance"] = 1.0 - doc["cosine_similarity"]  # Dystans kosinusowy
            retrieved_docs.append(doc)
    
    return retrieved_docs

In [ ]:
# wyszukujemy tutaj dokument dla zapytania identycznego z jednym z dokumentów z bazy
# Odnaleziony dokument ma cosine_similarity = 1.0, a więc potwierdzamy, że to co FAISS zwrócił jako distance, w rzeczywistości jest podobieństwem kosinusowym
retrieve_documents_vector("Laravel to framework aplikacji webowych z elegancką składnią dla PHP, ułatwiający typowe zadania programistyczne.", documents, embedding_model, index, top_k=10)

In [ ]:
retrieve_documents_vector("jakie modele stworzyło OpenAI?", documents, embedding_model, index, top_k=10)

In [ ]:
retrieve_documents_vector("Czym mogę stworzyć bezpieczne połączenie przez Internet?", documents, embedding_model, index, top_k=10)

#### Generowanie odpowiedzi na podstawie wyszukanych dokumentów

In [ ]:
def generate_answer_vector(query, retrieved_docs):
    """
    Generuje odpowiedź na podstawie zapytania i wyszukanych dokumentów.
    
    Args:
        query: Zapytanie użytkownika
        retrieved_docs: Lista wyszukanych dokumentów z ocenami podobieństwa
        
    Returns:
        Wygenerowana odpowiedź
    """
    # Przygotowanie kontekstu dla modelu LLM
    context = "\n\n".join([
        f"Dokument {doc['id']}: {doc['content']}\n"
        f"Źródło: {doc['source']}\n"
        f"Ocena podobieństwa: {doc['cosine_similarity']:.2f}"
        for doc in retrieved_docs
    ])
    
    # Instrukcje dla modelu
    system_prompt = f"""
    Jesteś pomocnym asystentem, który odpowiada na pytania użytkownika w oparciu o dostarczone konteksty.
    Twoim zadaniem jest:
    1. Przeanalizować dostarczone dokumenty i wybrać te, które są najbardziej istotne dla pytania.
    2. Sformułować zwięzłą, ale kompletną odpowiedź opartą na tych dokumentach.
    3. Gdy informacje są niepełne lub niepewne, zaznaczyć to w odpowiedzi.
    4. Podać źródła użytych informacji w formie odnośników [1], [2], itp. na końcu odpowiedzi, gdzie numer odpowiada ID dokumentu.
    
    Odpowiedź powinna być napisana w języku polskim, prosta do zrozumienia i dobrze zorganizowana.
    """
    
    prompt = f"""
    Pytanie: {query}
    ---
    Kontekst:
    {context}
    """
    
    # Przygotowanie wiadomości dla API
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]
    
    # Wywołanie API
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0.2,  # Niska temperatura dla bardziej deterministycznych odpowiedzi
        max_tokens=10000  # Limit długości odpowiedzi
    )
    
    return response.choices[0].message.content

#### Funkcja główna łącząca wszystkie komponenty

In [ ]:
def rag_system_vector(query, top_k=3):
    """
    Kompletny system RAG oparty o bazę wektorową.
    
    Args:
        query: Zapytanie użytkownika
        documents: Lista dokumentów z metadanymi
        top_k: Liczba najbardziej podobnych dokumentów do wykorzystania
        
    Returns:
        Wygenerowana odpowiedź
    """
    print(f"Zapytanie: {query}")
    print("Inicjalizacja bazy wektorowej...")
    
    # Wyszukanie podobnych dokumentów
    print("Wyszukiwanie podobnych dokumentów...")
    start_time = time.time()
    retrieved_docs = retrieve_documents_vector(query, documents, embedding_model, index, top_k)
    retrieval_time = time.time() - start_time
    print(f"Znaleziono {len(retrieved_docs)} podobnych dokumentów w {retrieval_time:.2f} sekund.")
    
    # Generowanie odpowiedzi
    print("Generowanie odpowiedzi...")
    start_time = time.time()
    answer = generate_answer_vector(query, retrieved_docs)
    generation_time = time.time() - start_time
    print(f"Odpowiedź wygenerowana w {generation_time:.2f} sekund.")
    
    # Informacje o źródłach
    print("\nŹródła informacji:")
    for doc in retrieved_docs:
        print(f"[{doc['id']}] (Ocena podobieństwa: {doc['cosine_similarity']:.2f}) - {doc['content']} ")
    
    return answer

### Testowanie systemu RAG opartego o bazę wektorową

In [ ]:
query_vector = "Czym jest deep learning i jak się różni od tradycyjnego uczenia maszynowego?"
answer_vector = rag_system_vector(query_vector, top_k=5)
display_answer(answer_vector)

In [ ]:
query_vector = "Jaki jest najnowszy model OpenAI i co potrafi?"
answer_vector = rag_system_vector(query_vector, top_k=5)
display_answer(answer_vector)

In [ ]:
query_vector = "Czym mogę stworzyć bezpieczne połączenie przez Internet?"
answer_vector = rag_system_vector(query_vector, top_k=5)
display_answer(answer_vector)